In [111]:
import pandas as pd
import numpy as np
import os


In [112]:
data_vulnerabilities = pd.read_json(
    "../smartbugs/dataset/vulnerabilities.json")


In [113]:
data_vulnerabilities.head()

,name,path,source,vulnerabilities
0,FibonacciBalance.sol,dataset/access_control/FibonacciBalance.sol,https://github.com/sigp/solidity-security-blog,"[{'lines': [31], 'category': 'access_control'}..."
1,arbitrary_location_write_simple.sol,dataset/access_control/arbitrary_location_writ...,https://smartcontractsecurity.github.io/SWC-re...,"[{'lines': [27], 'category': 'access_control'}]"
2,incorrect_constructor_name1.sol,dataset/access_control/incorrect_constructor_n...,https://github.com/trailofbits/not-so-smart-co...,"[{'lines': [20], 'category': 'access_control'}]"
3,incorrect_constructor_name2.sol,dataset/access_control/incorrect_constructor_n...,https://smartcontractsecurity.github.io/SWC-re...,"[{'lines': [18], 'category': 'access_control'}]"
4,incorrect_constructor_name3.sol,dataset/access_control/incorrect_constructor_n...,https://smartcontractsecurity.github.io/SWC-re...,"[{'lines': [17], 'category': 'access_control'}]"


In [114]:
data_vulnerabilities.vulnerabilities[0]


[{'lines': [31], 'category': 'access_control'},
 {'lines': [38], 'category': 'access_control'}]

In [115]:
all_categories = []
for idx, row in data_vulnerabilities.iterrows():
    for i in row['vulnerabilities']:
        # print(i)
        all_categories.append(i['category'])
unique_categories = set(all_categories)

In [116]:
unique_categories

{'access_control',
 'arithmetic',
 'bad_randomness',
 'denial_of_service',
 'front_running',
 'other',
 'reentrancy',
 'short_addresses',
 'time_manipulation',
 'unchecked_low_level_calls'}

In [117]:
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
label_encoded_category = label_encoder.fit_transform(list(unique_categories))

In [118]:
def parse_clean_Data(data, filename):
    indexes_to_drop = []
    for i, d in enumerate(data):
        # Convert bytes to string
        data[i] = data[i].decode("utf-8")
        d = d.decode("utf-8")
        # print(type(d))
        if d.startswith("/*"):
            indexes_to_drop.append(i)
            data[i] = ""
            # pass
        elif d.startswith("*") or d.__contains__("@"):
            indexes_to_drop.append(i)
            data[i] = ""
            # pass
        elif d.startswith(" */"):
            indexes_to_drop.append(i)
            data[i] = ""
            pass
        elif d.startswith("//"):
            indexes_to_drop.append(i)
            data[i] = ""
            pass
        elif d.__contains__("//"):
            data[i] = data[i].split("//")[0]
    # print(indexes_to_drop)
    # result = {f"{filename}":data}
    return data
    # data = data.dropna()
    # data = data.reset_index(drop=True)
    # return data


In [119]:
list_of_dataset=os.listdir("../smartbugs/dataset/")
#Loop to check location is directory or not
final_output = {}
for entry in list_of_dataset:
    # Create full path
    fullpath = os.path.join("../smartbugs/dataset/", entry)
    # If entry is a directory then get the list of files in this directory 
    if os.path.isdir(fullpath):
        print(entry)
        filenames_list = os.listdir(fullpath)
        for filename in filenames_list:
            if(filename!="README.md"):
            # print(filename)
            # Read the file content and print it
                with open(os.path.join(fullpath, filename), 'rb') as f:
                    file_Data = f.read().splitlines()
                    # final_output.append(parse_clean_Data(file_Data, filename))
                    final_output[filename] = parse_clean_Data(file_Data, filename)
                # print()
# print(final_output)
    

access_control
arithmetic
bad_randomness
denial_of_service
front_running
other
reentrancy
short_addresses
time_manipulation
unchecked_low_level_calls


In [120]:
all_filenames = []
all_codes = []
all_categories = []
for filename,v in final_output.items():
    temp_V= list(enumerate(v))
    #apply lambda function on list of tuples
    temp_V = list(map(lambda x: (x[0]+1, x[1]), temp_V))
    #get the index for the value in the dataframe
    # print(filename)
    index = data_vulnerabilities[data_vulnerabilities['name'] == filename].index[0]
    for (j,k) in temp_V:    
        for vuln in data_vulnerabilities.vulnerabilities[index]:
            # print(i)
            if j== vuln["lines"][0]:
                all_categories.append(vuln['category'])
                all_codes.append(k)
                all_filenames.append(filename)
            else:
                all_categories.append("No Vulnerability")
                # all_codes.append(temp_V)
                # all_filenames.append(filename)
                all_codes.append(k)
                all_filenames.append(filename)
    # print(index)


In [121]:
final_df = pd.DataFrame(list(zip(all_filenames, all_codes, all_categories)),columns=['filename', 'code', 'category'])

In [125]:
df1=final_df[final_df["category"]=="No Vulnerability"]
df1 = df1.drop_duplicates(subset=['code'], keep='first')
df1

,filename,code,category
0,arbitrary_location_write_simple.sol,,No Vulnerability
6,arbitrary_location_write_simple.sol,pragma solidity ^0.4.25;,No Vulnerability
8,arbitrary_location_write_simple.sol,contract Wallet {,No Vulnerability
9,arbitrary_location_write_simple.sol,uint[] private bonusCodes;,No Vulnerability
10,arbitrary_location_write_simple.sol,address private owner;,No Vulnerability
...,...,...,...
35972,mishandled.sol,function withdrawBalance() {,No Vulnerability
35987,unchecked_return_value.sol,contract ReturnValue {,No Vulnerability
35989,unchecked_return_value.sol,function callchecked(address callee) public {,No Vulnerability
35990,unchecked_return_value.sol,require(callee.call());,No Vulnerability


In [124]:
#import TfidfVectorizer from sklearn
from sklearn.feature_extraction.text import TfidfVectorizer
#initialize TfidfVectorizer 
tfidf_vectorizer=TfidfVectorizer(use_idf=True)
#convert the text into a matrix of TF-IDF features
tfidf_vectorizer_vectors=tfidf_vectorizer.fit_transform(final_df['code']).todense()

In [41]:
tfidf_vectorizer_vectors.shape

(35998, 2147)

In [43]:
from sklearn.preprocessing import LabelEncoder
l_e = LabelEncoder()
final_df['category'] = l_e.fit_transform(final_df['category'])

In [44]:
final_df

,filename,code,category
0,arbitrary_location_write_simple.sol,,0
1,arbitrary_location_write_simple.sol,,0
2,arbitrary_location_write_simple.sol,,0
3,arbitrary_location_write_simple.sol,,0
4,arbitrary_location_write_simple.sol,,0
...,...,...,...
35993,unchecked_return_value.sol,function callnotchecked(address callee) publ...,0
35994,unchecked_return_value.sol,,0
35995,unchecked_return_value.sol,callee.call();,10
35996,unchecked_return_value.sol,},0


In [49]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(tfidf_vectorizer_vectors, final_df['category'], test_size=0.2, random_state=42)


In [56]:
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression(C=0.0001,max_iter=10000000)
lr.fit(X_train, y_train)

LogisticRegression(C=0.0001, max_iter=10000000)

In [59]:
from sklearn.metrics import accuracy_score,classification_report
accuracy_score(y_test, lr.predict(X_test))
print(classification_report(y_test, lr.predict(X_test)))
# lr.predict(tfidf_vectorizer_vectors)

              precision    recall  f1-score   support

           0       0.99      1.00      1.00      7157
           1       0.00      0.00      0.00         2
           2       0.00      0.00      0.00         6
           3       0.00      0.00      0.00         5
           4       0.00      0.00      0.00         1
           5       0.00      0.00      0.00         2
           6       0.00      0.00      0.00         1
           7       0.00      0.00      0.00         5
           9       0.00      0.00      0.00         2
          10       0.00      0.00      0.00        19

    accuracy                           0.99      7200
   macro avg       0.10      0.10      0.10      7200
weighted avg       0.99      0.99      0.99      7200



c:\Users\vatsa\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\vatsa\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\vatsa\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
